In [2]:
import pandas as pd
from coypu.utils.query import Query
from coypu.utils.credentials import *
from query.queries import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


coypu.utils


In [15]:
cmemc_query_imp = Query(client_url_imp, client_id_imp, client_secret_imp , 'oauth')

    
# fuseki_infai = Query(fuseki_endpoint_infai, fuseki_user_infai, fuseki_pw_infai, 'basic')
fuseki_infai = Query("https://skynet.coypu.org/coypu-internal/", "katherine", "0CyivAlseo", 'basic')

fdquery = Query("http://localhost:5002/sparql", is_fdq=True)

#cmemc_query_tib = Query(client_url_tib_1, client_id_tib_1, client_secret_tib_1, 'oauth')



In [16]:
fuseki_infai.headers, fuseki_infai.headers
print(query_0)
fuseki_infai.post_query(query_0)


PREFIX coy: <https://schema.coypu.org/global#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX time: <http://www.w3.org/2006/time#> 

PREFIX wb: <http://worldbank.org/>
PREFIX wbi: <http://worldbank.org/Indicator/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX geo: <https://www.geonames.org/ontology#>

SELECT * WHERE{
    ?subject ?predicate ?object
}LIMIT 10

Passed Query Description: 200
Total time taken by post_query function: 1.2713184356689453


In [17]:
fuseki_infai.response.content

b'subject,predicate,object\r\nhttps://data.coypu.org/nearby/1015559/BRA0027793,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://schema.coypu.org/global#NearbyInfrastructure\r\nhttps://data.coypu.org/nearby/1015559/BRA0027793,https://schema.coypu.org/global#isNearby,https://data.coypu.org/infrastructure/BRA0027793\r\nhttps://data.coypu.org/nearby/1015559/BRA0027793,https://schema.coypu.org/global#hasDistance,51\r\nhttps://data.coypu.org/nearby/1015559/BRA0027793,http://www.w3.org/2000/01/rdf-schema#label,Nearby Infrastructure - GDACS Event 1015559\r\nhttps://data.coypu.org/event/gdacs/1015559,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://schema.coypu.org/global#Disaster\r\nhttps://data.coypu.org/event/gdacs/1015559,https://schema.coypu.org/global#hasNearbyInfrastructure,https://data.coypu.org/nearby/1015559/BRA0027793\r\nhttps://data.coypu.org/event/gdacs/1015559,https://schema.coypu.org/global#hasNearbyInfrastructure,https://data.coypu.org/nearby/1015559/BRA0029796\r\n

In [ ]:
print(query_1_fdq_desc)
print(query_1_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_1_fdq_ex))
fdquery.post_query(query_1_fdq_ex)
fdquery.to_save('../../query_results/', filename='query_1_fdq_ex')

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_1_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_1_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

query_1_fdq_ex_sql = """SELECT country_code, year, AVG(value) as carbon_emission, COUNT(disaster) AS no_of_disasters
FROM `query_1_fdq_ex`
WHERE year=year_dis
GROUP By country_code, year
Order By  year ASC
"""

df = pd.read_sql(query_1_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [ ]:
def carbon_emission_group(carbon_emission):
    if carbon_emission/1000000 < 0.25:
        return '0.00-0.25'
    elif carbon_emission/1000000 >= 0.25 and carbon_emission/1000000 < 0.5:
        return '0.25-0.5'
    elif carbon_emission/1000000 >= 0.5 and carbon_emission/1000000 < 0.75:
        return '0.50-0.75'
    else:
        return '> 0.75'
    

df = pd.read_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8')
df['carbon_emission(1e6)'] = df['carbon_emission'].apply(carbon_emission_group)

df.head(10)

In [ ]:

fig = px.scatter_geo(df, locations="country_code", color="carbon_emission(1e6)",
                     hover_data=['carbon_emission'], size="no_of_disasters",
                     animation_frame="year",
                     projection="equirectangular", scope='europe')
fig.update_layout(title_text='Carbon emission and the number of disaster events for various countries across years',\
                  title_font_color="red", title_font_family="Times New Roman", title_font_size=25)
fig.write_html('../../query_results/query_1_fdq_ex_sql_new.html')
fig.show()